# Weaviate Experiment Example

## Installations

In [1]:
# !pip install --quiet --force-reinstall prompttools
# !pip install weaviate-client

First, you need to connect to a Weaviate client. One easy option is to connect to a local instance launched with Docker.

Please see their official documentation for details: https://weaviate.io/developers/weaviate/tutorials/connect

In [2]:
import weaviate

client = weaviate.Client("http://localhost:8080/")  # This can be replaced by other clients

## Run an experiment

### Working with data already exists in Weaviate

First, we will show how you can run experiment if you already have datat that exists in Weaviate. You can test different queries to see how the responses differ.

You can also skip to the next section if you would like to insert data at runtime.

In [3]:
# Can skip if you already have data in Weaviate

# Create class object
class_name = "Question"

# Clearing existing
try:
    client.batch.delete_objects(class_name=class_name, where={})
except Exception:
    pass

try:
    client.schema.delete_class(class_name)
except Exception:
    pass

class_obj = {
    "class": class_name,
    "vectorizer": "text2vec-contextionary",
    "moduleConfig": {"text2vec-contextionary": {"vectorizeClassName": "false"}},
}

client.schema.create_class(class_obj)

We insert data in the following cell.

In [4]:
# Can skip if you already have data in Weaviate
data = [
    {"Category": "science", "Question": "How many hydrogens are in H2O?", "Answer": "2"},
    {"Category": "math", "Question": "2+2", "Answer": "4"},
    {"Category": "math", "Question": "Is 13 a prime number?", "Answer": "Yes"},
    {"Category": "geography", "Question": "Which continent is India in?", "Answer": "Asia"},
    {"Category": "geography", "Question": "Which continent is China in?", "Answer": "Asia"},
    {"Category": "geography", "Question": "Which continent is USA in?", "Answer": "North America"},
    {"Category": "geography", "Question": "Which is the largest state in the USA?", "Answer": "Alaska"},
    {"Category": "geography", "Question": "Which is the most populous state in the USA?", "Answer": "California"},
]

with client.batch(batch_size=100) as batch:
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        client.batch.add_data_object(
            properties,
            "Question",
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8


#### Building differnt queries

In this first example, we will demonstrate how you can test different queries on the same existing dataset.

Given that there are many options on how a near text search operator and filters can be constructed. We ask that you define query functions and pass that into the experiment in the form of a dictionary `{name: callable_function}`, as shown below.

Note: there is a default query function if you aren't interested in testing different querying methods.

In [5]:
def default_query_builder(
    client: weaviate.Client,
    class_name: str,
    property_names: list[str],
    text_query: str,
):
    near_text_search_operator = {"concepts": [text_query]}
    return client.query.get(class_name, property_names).with_near_text(near_text_search_operator).with_limit(limit=3)


def custom_query_builder(
    client: weaviate.Client,
    class_name: str,
    property_names: list[str],
    text_query: str,
):
    near_text_search_operator = {
        "concepts": [text_query],
        "distance": 0.6,
        "moveAwayFrom": {"concepts": ["where geography "], "force": 0.45},
    }
    return client.query.get(class_name, property_names).with_near_text(near_text_search_operator).with_limit(limit=3)


# A dictionary of the name of query builder and the corresponding callable function.
query_builders = {"default": default_query_builder, "custom": custom_query_builder}

We will define our queries and run the experiment!

In [6]:
from prompttools.experiment import WeaviateExperiment

use_existing_data = True
property_names = ["category", "question", "answer"]  # Specify what property you would like to query
text_queries = ["Hydrogen", "USA"]  # Your text queries that will be passed in to query builder to create query


experiment = WeaviateExperiment(
    client=client,
    class_name=class_name,
    use_existing_data=use_existing_data,
    property_names=property_names,
    text_queries=text_queries,
    query_builders=query_builders,
)
experiment.run()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


As you can see from the results, the "custom" queries try to stay away from geography topics in its responses (unless it is highly relevant), whereas the first one is agnostic.

In [7]:
experiment.visualize()

,text_query,query_builder_name,top objs
0,Hydrogen,default,"[{'answer': '2', 'category': 'science', 'question': 'How many hydrogens are in H2O?'}, {'answer': 'Alaska', 'category': 'geography', 'question': 'Which is the largest state in the USA?'}, {'answer': 'Asia', 'category': 'geography', 'question': 'Which continent is India in?'}]"
1,USA,default,"[{'answer': 'Alaska', 'category': 'geography', 'question': 'Which is the largest state in the USA?'}, {'answer': 'California', 'category': 'geography', 'question': 'Which is the most populous state in the USA?'}, {'answer': 'North America', 'category': 'geography', 'question': 'Which continent is USA in?'}]"
2,Hydrogen,custom,"[{'answer': '2', 'category': 'science', 'question': 'How many hydrogens are in H2O?'}]"
3,USA,custom,"[{'answer': 'Alaska', 'category': 'geography', 'question': 'Which is the largest state in the USA?'}, {'answer': 'California', 'category': 'geography', 'question': 'Which is the most populous state in the USA?'}, {'answer': 'North America', 'category': 'geography', 'question': 'Which continent is USA in?'}]"


### Adding data into Weaviate during the experiment

If you want to test different vectorizers (embedding functions), you may want to add data into Weaviate during the experiment.

In the example below, we will be inserting the same data as we were doing in the above section.

Define the `vectorizer` and corresponding `moduleConfig` in a list of tuples. Although the example here has 1, you can define multiple here if you want to compare multiple vectorizers.

In [8]:
vectorizers_and_moduleConfigs = [
    (
        "text2vec-contextionary",
        {  # This runs on CPU, but you can swap to `huggingface` or `openai` as well
            "text2vec-contextionary": {"vectorizeClassName": "false"}
        },
    ),
]

We need to definte the properties for the data that we are about to insert. You can customize how and whether each property is vectorized if you wish.

In [9]:
property_definitions = [
    {
        "name": "category",
        "dataType": ["text"],
    },
    {
        "name": "question",
        "dataType": ["text"],
    },
    {
        "name": "answer",
        "dataType": ["text"],
    },
]

By default, Weaviate uses cosine similarity as the distance function. We can compare it with other ones.

In [10]:
distance_metrics = ["cosine", "l2-squared"]

In [11]:
class_name = "Question"

# Clearing existing class object
try:
    client.batch.delete_objects(class_name=class_name, where={})
except Exception:
    pass

try:
    client.schema.delete_class(class_name)
except Exception:
    pass

In [12]:
use_existing_data = False
property_names = ["question", "answer"]  # Specify what property you would like to query
text_queries = ["continent Asia"]  # Your text queries that will be passed in to query builder to create query


experiment = WeaviateExperiment(
    client=client,
    class_name=class_name,
    use_existing_data=use_existing_data,
    property_names=property_names,
    text_queries=text_queries,
    vectorizers_and_moduleConfigs=vectorizers_and_moduleConfigs,
    property_definitions=property_definitions,
    data_objects=data,
    distance_metrics=distance_metrics,
)
experiment.run()

In [13]:
experiment.visualize()

,vectorIndexConfig,top objs
0,{'vectorIndexConfig': {'distance': 'cosine'}},"[{'answer': 'Asia', 'question': 'Which continent is China in?'}, {'answer': 'Asia', 'question': 'Which continent is India in?'}, {'answer': 'North America', 'question': 'Which continent is USA in?'}]"
1,{'vectorIndexConfig': {'distance': 'l2-squared'}},"[{'answer': 'Asia', 'question': 'Which continent is China in?'}, {'answer': 'Asia', 'question': 'Which continent is India in?'}, {'answer': 'North America', 'question': 'Which continent is USA in?'}]"


## Evaluate the model response

For now, please refer to the ChromaDB example for possible evaluation methods (i.e. ranking correlation). Other examples show case how you can evaluate with LLMs.